In [1]:

import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img,img_to_array

from random import shuffle
from tqdm import tqdm
Train_DIR=r'D:\train'
Test_DIR=r'C:\Users\Anuj\Desktop\work\test1'
IMG_SIZE=50
LR= 1e-3
modelname='dvc1'
def label_img(img):
    word_label=img.split('.')[-3]
    if word_label=='cat' :return[1,0]
    elif word_label=='dog':return[0,1]
    

In [3]:
def creat_train_data():
    training_data=[]
    for img in tqdm(os.listdir(Train_DIR)):
        label=label_img(img)
        path=os.path.join(Train_DIR,img)
        img=load_img(path,target_size=(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('tain_data.npy',training_data)
    return training_data

In [4]:
def process_test_data():
    testing_data=[]
    for img in tqdm(os.listdir(Test_DIR)):
        path=os.path.join(Test_DIR,img)
        img_num=img.split('.')[0]
        img=load_img(path,target_size=(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy',testing_data)
    return testing_data


In [5]:
train_data=creat_train_data()
print(train_data)

100%|██████████| 25000/25000 [05:42<00:00, 72.93it/s] 
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,BatchNormalization,Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.constraints import max_norm
from tensorflow.python.keras.utils.np_utils import to_categorical

from keras import optimizers








train=train_data[:-500]
test=train_data[-500:]
x=np.array([i[0]for i in train])
print(x)
test_x=np.array([i[0]for i in test])
y=([i[1]for i in train])
test_y=np.array([i[1]for i in test])

x=x.astype('float32')
test_x=test_x.astype('float32')
x=x/255.0
test_x=test_x/255.0


model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE,IMG_SIZE,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())



model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())



model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=max_norm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=max_norm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Dense(2))
model.add(Activation('softmax'))


epochs = 10
optimizer = 'adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model.summary())
print(len(x))
print(len(y))



Using TensorFlow backend.


[[[[ 84  83  79]
   [ 86  78  76]
   [ 87  82  79]
   ...
   [120 120 120]
   [103 103 103]
   [108 108 108]]

  [[ 79  78  74]
   [ 85  77  75]
   [ 93  88  85]
   ...
   [114 113 119]
   [102 102 102]
   [ 99  99  99]]

  [[ 93  88  85]
   [ 85  80  77]
   [ 95  90  87]
   ...
   [112 107 111]
   [ 97  95  96]
   [ 98  96  97]]

  ...

  [[  4   6   5]
   [ 12   8   9]
   [  8   8   8]
   ...
   [224 217 211]
   [207 197 195]
   [171 161 159]]

  [[ 65  65  53]
   [ 54  47  37]
   [ 33  33  25]
   ...
   [180 174 176]
   [154 144 145]
   [135 125 126]]

  [[ 13  12  10]
   [ 20  14  14]
   [ 14  12  13]
   ...
   [218 213 209]
   [161 157 156]
   [154 150 149]]]


 [[[132 121 101]
   [137 124 115]
   [101  85  72]
   ...
   [ 95  82  76]
   [ 73  59  50]
   [ 51  36  31]]

  [[159 140 142]
   [169 152 142]
   [ 58  46  32]
   ...
   [152 134 130]
   [ 65  50  45]
   [ 92  78  67]]

  [[ 51  48  33]
   [137 133 121]
   [121 101  90]
   ...
   [ 82  73  66]
   [ 75  61  48]
   [118 102

In [77]:
model.fit(x, y, validation_data=(test_x, test_y), epochs=epochs, batch_size=64)


ValueError: Error when checking input: expected conv2d_36_input to have 4 dimensions, but got array with shape (50, 50, 73500)

In [28]:
scores = model.evaluate(test_x, test_y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

 - 0s 386us/sample - loss: 0.3689 - accuracy: 0.8680
Accuracy: 86.80%


In [29]:
model.save("anuj1.h5")

In [30]:
model.save_weights("anuj1.h5")

In [31]:
with open('Subfile.csv','w')as f:
    f.write('id , label\n')


In [32]:

test_data= process_test_data()


        
    
    
   

100%|██████████| 12499/12499 [00:30<00:00, 410.06it/s]


In [64]:

    c=np.array([i[0]for i in test_data])
    c=c.astype('float32')
    c=c/255.0
    for l in range(0,10):
        model_out=model.predict(c)[l]
        print(model_out)
   
        

[0.00376901 0.996231  ]
[9.997359e-01 2.641069e-04]
[0.97534806 0.02465197]
[0.01437408 0.9856259 ]
[0.3522278  0.64777213]
[9.9978763e-01 2.1239142e-04]
[0.993145   0.00685504]
[0.04391043 0.9560896 ]
[0.04046479 0.9595352 ]
[9.999049e-01 9.516041e-05]
